In [1]:
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtWidgets import (
    QApplication,
    QWidget,
    QPushButton,
    QGridLayout,
    QComboBox,
)
from typing import Optional
from functools import partial
import sys

In [2]:
qapp = QApplication.instance()
if qapp is None:
    qapp = QApplication([])

In [3]:
class Context:

    def get_base64_png(self, rel_path: str, chunk_size: Optional[int] = 1024) -> str:
        from os.path import abspath, exists
        from base64 import b64encode

        file_path = abspath(rel_path)
        try:
            assert exists(file_path)
        except AssertionError:
            file_path = abspath('./ey-1-print-LC1.png')

        with open(file_path, 'rb') as fp:
            b64_str = b64encode(fp.read()).decode('utf-8')
            
            return f'data:image/png;base64,{b64_str}'

In [ ]:
class Coupon:

    def __init__(self) -> None:

        self.w = 8.0
        self.h = 4.0
        self.Ep = 10e6

In [4]:
class WebView(QWidget):

    def __init__(self, ctx: object) -> None:
        super().__init__()

        self.ctx = ctx
        hole_idx_widget = QComboBox(parent=self, objectName='Hole Index')
        hole_idx_widget.addItems(['1', '2', '3'])
        load_case_widget = QComboBox(parent=self, objectName='report load case')
        load_case_widget.addItems(['LC1', 'LC2'])
        self.print_batch_results = self.repair_batch_results = {
            'LC1': {
                '+mid': {idx: None for idx in range(3)},
            },
            'LC2': {
                '+mid': {idx: None for idx in range(3)},
            },
        }

        model = 'mlh'
        load_case_list = [load_case_widget.itemText(i) for i in range(load_case_widget.count())]
        num_holes = hole_idx_widget.count()
        self.get_detailed_report(model, load_case_list, num_holes, is_repair=True)
    
    def get_detailed_report(self, model: str, load_case_list: list, num_holes: int, is_repair: Optional[bool] = False) -> None:

        from PyQt5.QtCore import Qt

        widget = QWidget(self, flags=Qt.Window)
        widget.setWindowTitle('Detailed Report')
        widget.show()
        layout = QGridLayout(widget)
        layout.setColumnStretch(1, 2)
        web_view = QWebEngineView(self)
        pdf_btn = QPushButton('PDF', self)
        widgets = (
            (web_view, 0, 0, 1, 2),
            (pdf_btn, 1, 0, 1, 1),
        )
        for widget, row, col, rowspan, colspan in widgets:
            layout.addWidget(widget, row, col, rowspan, colspan)
        
        html = self.get_html(model, load_case_list, num_holes, is_repair)
        web_view.setHtml(html)
        widget.resize(web_view.page().sizeHint())
      
        pdf_btn.clicked.connect(partial(self.export_webpage, web_view))

    def get_html(self, model: str, load_case_list: list, num_holes: int, is_repair: bool) -> str:

        print_summary_table = self.get_summary_table(model, num_holes, is_repair=is_repair)
        if is_repair:
            repair_summary_table = self.get_summary_table(model, num_holes, repair_results=True, is_repair=True)
        else:
            repair_summary_table = '<table></table>'
        
        coupon_table = self.get_coupon_html_table(model, num_holes, is_repair)  # w, h, th, E, nu, x, y, d, df, Ef, csk
        applied_factor_table = self.get_applied_factor_html_table(model, is_repair)  # t/d, csk, nf, p
        matl_cf_table = self.get_matl_cf_html_table(is_repair)  # fg, ft, fsf, fsc

        load_case = self.findChild(QComboBox, 'report load case').currentText()
        if is_repair:
            results_html = f"""
                <h3>6.0 Print {load_case} Critical Fatigue Stress Quantities</h3>
                {self.get_report_html_contents(model, section=6, num_holes=num_holes, load_case=load_case)}
                <h3>7.0 Repair {load_case} Critical Fatigue Stress Quantities</h3>
                {self.get_report_html_contents(model, section=7, num_holes=num_holes, load_case=load_case, repair=True)}
                """
        else:
            results_html = f"""
                <h3>6.0 {load_case} Critical Fatigue Stress Quantities</h3>
                {self.get_report_html_contents(model, section=6, num_holes=num_holes, load_case=load_case)}
                """

        html = f"""
            <!DOCTYPE html>
            <head>
                <meta name="report" content="width=device-width, initial-scale=1">
                <style>

                    body {{
                        margin: 0;
                        background-color: #f1f1f1;
                        font-family: Arial, Helvetica, sans-serif;
                    }}

                    #navbar, #sidenav {{
                        background-color: #5084BC;
                        position: fixed;
                        top: 0;
                        width: 100%;
                        display: block;
                        transition: top 0.3s;
                    }}

                    #navbar a, #sidenav a {{
                        display: inline-block;
                        color: #f2f2f2;
                        text-align: center;
                        padding: 8px 16px;
                        text-decoration: none;
                        font-size: 16px;
                    }}
                    
                    #sidenav a {{
                        color: #000;
                    }}

                    #sidenav a:hover {{
                        background-color: #ddd;
                    }}

                    .body::-webkit-scrollbar {{
                        display: none;
                    }}

                    .button {{
                        display: inline-block;
                        padding: 8px 16px;
                        overflow: hidden;
                        text-decoration: none;
                        color: #000;
                        background-color: inherit;
                        cursor: pointer;
                        white-space: nowrap;
                        outline: none;
                        border: none;
                    }}
                    
                    .button:hover {{
                        background-color: #ddd;
                        color: black;
                    }}

                    #summary {{
                        font-family: Roboto;
                        font-size: 16px;
                        border-collapse: collapse;
                        margin-left: 10%;
                        margin-right: 10%;
                    }}

                    #summary td, #summary th {{
                        border: 1px solid #ddd;
                        padding: 8px;
                    }}

                    #summary tr:nth-child(even) {{ background-color: #f2f2f2; }}
                    #summary tr:hover {{ background-color: #ddd; }}
                    
                    #summary th {{
                        padding-top: 12px;
                        padding-bottom: 12px;
                        text-algin: left;
                        background-color: #5084BC;
                        color: white;
                    }}

                </style>
            </head>
            <body class="body">

            <div id="sidenav" style="display:none;">
                <button id="hide" class="button" onclick="hide_sidenav()">&times;</button><br>
                <a href="#1" onclick="hide_sidenav()">Section 1.0</a>
                <a href="#2" onclick="hide_sidenav()">Section 2.0</a>
                <a href="#3" onclick="hide_sidenav()">Section 3.0</a>
            </div>
            
            <div id="main">
              
                <div id="navbar">
                    <button id="show" class="button" onclick="show_sidenav()">&#9776;</button>
                    <a>Title</a>
                    <a>Other</a>
                </div>
              
                <div id="report" style="margin-top: 50px;">
                    <h3 id="1">1.0 Fatigue Results Summary</h3>
                    {print_summary_table}
                    <br>
                    {repair_summary_table}
                    <h3 id="2">2.0 Coupon Geometry</h3>
                    {coupon_table}
                    <br>
                    {applied_factor_table}
                    <h3 id="3">3.0 Material Correction Factors</h3>
                    {matl_cf_table}
                    <h3 id="3">4.0 Fatigue Loads</h3>
                    {results_html}
                    <h3 id="A">Appendix A - K<sub>t</sub>\u03c3 Plots</h3>
                    <h3 id="B">Appendix B - Coupon Constraints</h3>
                    <h3 id="C">Appendix C - Fringe Plots</h3>
                    <h3 id="D">Appendix D - Allowable Tracking</h3>
              </div>
            </div>

            <script>
              var prevScrollpos = window.pageYOffset;
              window.onscroll = function() {{
                var currentScrollPos = window.pageYOffset;
                if (prevScrollpos > currentScrollPos) {{
                  document.getElementById("navbar").style.top = "0";
                }} else {{
                  document.getElementById("navbar").style.top = "-50px";
                }}
                prevScrollpos = currentScrollPos;
              }}
              
              function show_sidenav() {{
                document.getElementById("main").style.marginLeft = "25%";
                document.getElementById("sidenav").style.width = "25%";
                document.getElementById("sidenav").style.display = "inline-block";
                document.getElementById("show").style.display = "none";
              }}

              function hide_sidenav() {{
                document.getElementById("main").style.marginLeft = "0%";
                document.getElementById("sidenav").style.display = "none";
                document.getElementById("show").style.display = "inline-block";
              }}
            </script>

          </body>
        """
        
        return html

    def get_summary_table(
            self, 
            model: str, 
            num_holes: int, 
            repair_results: Optional[bool] = False, 
            is_repair: Optional[bool] = False,
            ) -> str:

        if repair_results:
            results = self.repair_batch_results
        else:
            results = self.print_batch_results
        
        if model == 'satellite-hole':
            hole_map = {1: 'Central Hole', 2: 'Rivet-a', 3: 'Rivet-b'}
        else:
            hole_map = {}

        cells = []
        for load_case, _results in results.items():
            row = 0
            
            for idx in _results.get('+mid', {}).keys():
                if row > 0:
                    html = f"""
                        <tr>
                            <td>{hole_map.get(idx + 1, idx + 1)}</td>
                            <td>{_results.get('kts', {}).get(idx, 0) / 1e3:0.3f}</td>
                            <td>{_results.get('location', {}).get(idx, 'Mid-Surface, 0 deg')}</td>
                            <td>{_results.get('kckt', {}).get(idx, 0):0.3f}</td>
                            <td>{_results.get('gradient ratio', {}).get(idx, 0):0.3f}</td>
                        </tr>"""
                        #     <td>{_results.get('ktdls', {}).get(idx, {}).get('kckt', 0) / 1e3:0.3f}</td>
                        #     <td>{_results.get('margin', {}).get(idx, 0):0.3f}</td>
                        # </tr>"""
                else:
                    html = f"""
                        <tr>
                            <td>{hole_map.get(idx + 1, idx + 1)}</td>
                            <td rowspan="{num_holes}">{load_case}</td>
                            <td rowspan="{num_holes}">TensorX</td>
                            <td>{_results.get('kts', {}).get(idx, 0) / 1e3:0.3f}</td>
                            <td>{_results.get('location', {}).get(idx, 'Mid-Surface, 0 deg')}</td>
                            <td>{_results.get('kckt', {}).get(idx, 0):0.3f}</td>
                            <td>{_results.get('gradient ratio', {}).get(idx, 0):0.3f}</td>
                        </tr>"""
                        #     <td>{_results.get('ktdls', {}).get(idx, {}).get('kckt', 0) / 1e3:0.3f}</td>
                        #     <td>{_results.get('margin', {}).get(idx, 0):0.3f}</td>
                        # </tr>"""

                cells.append(html)
                row += 1
        
        if model == 'notch':
            hole_col = 'Notch Number'
        else:
            hole_col = 'Hole Number'
        
        if is_repair and repair_results:
            header = 'Repair'
        elif is_repair and not repair_results:
            header = 'Print'
        else:
            header = ''
        table = f"""
            <table id="summary">
                <tr>
                    <th colspan="9">{header}</th>
                </tr>
                <tr>
                    <th>{hole_col}</th>
                    <th>Load Case</th>
                    <th>Analysis Method</th>
                    <th>Kt\u03c3, ksi</th>
                    <th>Location</th>
                    <th>K<sub>c</sub>/K<sub>t</sub></th>
                    <th>Gradient Ratio</th>
                </tr>
                {''.join(cells)}
            </table>
          """
        #           <th>KtDLS, ksi</th>
        #           <th>Margin of Safety</th>
        #         </tr>
        
        return table
    
    def get_html_table_from_dict(self, coupon_dict: dict, num_holes: int) -> str:
        """
        Returns html table from dictionary with column, value(s) pairs.
        
        Parameters
        ----------
        coupon_dict : dict
            Dictionary of column names, values.
        num_holes: int
            Number of holes.
        
        Returns
        -------
        str
        
        """

        table_html = ['<table id=#summary>', ]
        for row in range(num_holes + 1):
            table_html.append('<tr>')
            for col in coupon_dict.keys():
                table_html.append('<tc>')

                if 0 < row <= 1:
                    try:
                        table_html.append(f'<td>{coupon_dict[col][row - 1]}</td>')
                    except TypeError:
                        table_html.append(f'<td rowspan="{num_holes}">{coupon_dict[col]}</td>')
                elif row > 1:
                    try:
                        table_html.append(f'<td>{coupon_dict[col][row - 1]}</td>')
                    except TypeError:
                        pass
                else:
                    table_html.append(f'<th>{col}</th>')
                table_html.append('</tc>')
            table_html.append('</tr>')
        table_html.append('</table>')

        return ''.join(table_html)

    def get_coupon_html_table(self, model: str, num_holes: int, is_repair: Optional[bool] = False) -> str:
        from PyQt5.QtWidgets import QDockWidget

        if model in ('single-hole', 'satellite-hole'):
            if model =='satellite-hole':
                hole_map = {0: 1, 1: 'Rivet-a', 2: 'Rivet-b'}
            else:
                hole_map = {idx: idx + 1 for idx in range(num_holes)}
            coupon = self.input_tab.coupon
            coupon_dict = {
                'Hole': {idx: hole_map[idx] for idx in range(num_holes)},
                'Width, in.': coupon.w,
                'Height, in.': coupon.h,
                '\u0395, Msi.': coupon.Ep / 1e6,
                '\u03bd': coupon.nu,
                'Thickness, in.': coupon.th,
                'X, in.': {idx: getattr(coupon, f'x{idx}') for idx in range(num_holes)},
                'Y, in.': {idx: getattr(coupon, f'y{idx}') for idx in range(num_holes)},
                'D, in.': {idx: getattr(coupon, f'd{idx}') for idx in range(num_holes)},
                'Csk. Depth, in.': {idx: getattr(coupon, f'csk{idx}') for idx in range(num_holes)},
                'D<sub>fastener</sub>, in.': {idx: getattr(coupon, f'df{idx}') for idx in range(num_holes)},
                '\u0395<sub>fastener</sub>, Msi.': {idx: getattr(coupon, f'Ef{idx}') / 1e6 for idx in range(num_holes)},
            }
            if is_repair:
                coupon_dict = {
                    **coupon_dict,
                    **{
                        'Thickness, in.': coupon.r_th,
                        'X, in.': {idx: getattr(coupon, f'x{idx}') for idx in range(num_holes)},
                        'Y, in.': {idx: getattr(coupon, f'y{idx}') for idx in range(num_holes)},
                        'D, in.': {idx: getattr(coupon, f'r_d{idx}') for idx in range(num_holes)},
                        'Csk. Depth, in.': {idx: getattr(coupon, f'r_csk{idx}') for idx in range(num_holes)},
                        'D<sub>fastener</sub>, in.': {idx: getattr(coupon, f'r_df{idx}') for idx in range(num_holes)},
                        '\u0395<sub>fastener</sub>, Msi.': {idx: getattr(coupon, f'r_Ef{idx}') / 1e6 for idx in range(num_holes)},
                    },
                }
            colspan = 4
        elif model == 'lug':
            coupon = self.input_tab.coupon
            coupon_dict = {
                'Hole': {0: 1},
                'Height, in.': coupon.h,
                '\u0395, Msi.': coupon.Ep / 1e6,
                '\u03bd': coupon.nu,
                'Thickness, in.': coupon.th,
                'X<sub>0</sub>, in.': coupon.x0,
                'Y<sub>0</sub>, in.': coupon.y0,
                'R<sub>0</sub>, in.': coupon.r0,
                'R<sub>i</sub>, in.': coupon.ri,
                'Csk. Depth, in.': {idx: getattr(coupon, f'csk{idx}') for idx in range(num_holes)},
                'R<sub>fastener</sub>, in.': {idx: getattr(coupon, f'rf{idx}') for idx in range(num_holes)},
                '\u0395<sub>fastener</sub>, Msi.': {idx: getattr(coupon, f'Ef{idx}') / 1e6 for idx in range(num_holes)},
            }
            if is_repair:
                coupon_dict = {
                    **coupon_dict,
                    **{
                        'Thickness, in.': coupon.r_th,
                        'X<sub>0</sub>, in.': coupon.x0,
                        'Y<sub>0</sub>, in.': coupon.y0,
                        'R<sub>0</sub>, in.': coupon.r0,
                        'R<sub>i</sub>, in.': coupon.r_ri,
                        'Csk. Depth, in.': {idx: getattr(coupon, f'r_csk{idx}') for idx in range(num_holes)},
                        'R<sub>fastener</sub>, in.': {idx: getattr(coupon, f'r_rf{idx}') for idx in range(num_holes)},
                        '\u0395<sub>fastener</sub>, Msi.': {idx: getattr(coupon, f'r_Ef{idx}') / 1e6 for idx in range(num_holes)},
                    },
                }
            colspan = 3
        elif model == 'mlh':
            coupon = self.coupons['print']
            coupon_dict = {
                'Width, in.': coupon.w,
                'Height, in.': coupon.h,
                'cx, in.': coupon.cx,
                'cy, in.': coupon.cy,
                '\u0395, Msi.': coupon.Ep / 1e6,
                '\u03bd': coupon.nu,
                'Thickness, in.': coupon.th,
                'X, in.': {idx: coupon.x[idx] for idx in range(num_holes)},
                'Y, in.': {idx: coupon.y[idx] for idx in range(num_holes)},
                'D, in.': {idx: coupon.d[idx] for idx in range(num_holes)},
                'Csk. Depth, in.': {idx: coupon.csk[idx] for idx in range(num_holes)},
                'D<sub>fastener</sub>, in.': {idx: coupon.df[idx] for idx in range(num_holes)},
                '\u0395<sub>fastener</sub>, Msi.': {idx: coupon.Ef[idx] / 1e6 for idx in range(num_holes)},
            }
            if is_repair:
                coupon = self.coupons['repair']
                coupon_dict = {
                    **coupon_dict,
                    **{
                        'Thickness, in.': coupon.th,
                        'X, in.': {idx: coupon.x[idx] for idx in range(num_holes)},
                        'Y, in.': {idx: coupon.y[idx] for idx in range(num_holes)},
                        'D, in.': {idx: coupon.d[idx] for idx in range(num_holes)},
                        'Csk. Depth, in.': {idx: coupon.csk[idx] for idx in range(num_holes)},
                        'D<sub>fastener</sub>, in.': {idx: coupon.df[idx] for idx in range(num_holes)},
                        '\u0395<sub>fastener</sub>, Msi.': {idx: coupon.Ef[idx] / 1e6 for idx in range(num_holes)},
                    },
                }
            colspan = 6
        elif model == 'notch':
            coupon = self.findChild(QDockWidget, 'coupon dock').widget()
            if coupon.name == 'FlatBottomNotch':
                coupon_dict = {
                    'Width, in.': coupon.geometry['w'],
                    'Height, in.': coupon.geometry['h'],
                    'cx, in.': coupon.geometry['cx'],
                    'cy, in.': coupon.geometry['cy'],
                    '\u0395, Msi.': coupon.geometry['modulus'] / 1e6,
                    '\u03bd': coupon.geometry['nu'],
                    'Thickness, in.': coupon.geometry['th'],
                    'X-center, in.': coupon.geometry['x'],
                    'Notch width, in.': 2 * coupon.geometry['a'],
                    'Notch depth, in.': coupon.geometry['b'],
                    'Notch radius': coupon.geometry['r'],
                }
            elif coupon.name == 'HyperbolicNotch':
                coupon_dict = {
                    'Width, in.': coupon.geometry['w'],
                    'Height, in.': coupon.geometry['h'],
                    'cx, in.': coupon.geometry['cx'],
                    'cy, in.': coupon.geometry['cy'],
                    '\u0395, Msi.': coupon.geometry['modulus'] / 1e6,
                    '\u03bd': coupon.geometry['nu'],
                    'Thickness, in.': coupon.geometry['th'],
                    'X-center, in.': coupon.geometry['x'],
                    'Notch depth, in.': coupon.geometry['b'],
                    'Notch radius, in.': coupon.geometry['r'],
                }
            elif coupon.name == 'VNotch':
                from numpy import pi

                coupon_dict = {
                    'Width, in.': coupon.geometry['w'],
                    'Height, in.': coupon.geometry['h'],
                    'cx, in.': coupon.geometry['cx'],
                    'cy, in.': coupon.geometry['cy'],
                    '\u0395, Msi.': coupon.geometry['modulus'] / 1e6,
                    '\u03bd': coupon.geometry['nu'],
                    'Thickness, in.': coupon.geometry['th'],
                    'X-center, in.': coupon.geometry['x'],
                    'Notch depth, in.': coupon.geometry['b'],
                    'Notch radius, in.': coupon.geometry['r'],
                    'Notch angle, deg': 180 * coupon.geometry['alpha'] / pi,
                }
            elif coupon.name == 'SymmetricNotch':
                coupon_dict = {
                    'Width, in.': coupon.geometry['w'],
                    'Height, in.': coupon.geometry['h'],
                    'cx, in.': coupon.geometry['cx'],
                    'cy, in.': coupon.geometry['cy'],
                    '\u0395, Msi.': coupon.geometry['modulus'] / 1e6,
                    '\u03bd': coupon.geometry['nu'],
                    'Thickness, in.': coupon.geometry['th'],
                    'X-center, in.': coupon.geometry['x'],
                    'Notch radius, in.': coupon.geometry['r'],
                    'Notch depth, in.': coupon.geometry['b'],
                }
            elif coupon.name == 'EllipticalNotch':
                coupon_dict = {
                    'Width, in.': coupon.geometry['w'],
                    'Height, in.': coupon.geometry['h'],
                    'cx, in.': coupon.geometry['cx'],
                    'cy, in.': coupon.geometry['cy'],
                    '\u0395, Msi.': coupon.geometry['modulus'] / 1e6,
                    '\u03bd': coupon.geometry['nu'],
                    'Thickness, in.': coupon.geometry['th'],
                    'X-center, in.': coupon.geometry['x'],
                    'Notch width, in.': 2 * coupon.geometry['a'],
                    'Notch depth, in.': coupon.geometry['b'],
                }
            else:
                raise Exception(f'Notch model {coupon.name} invalid.')

        else:
            raise Exception(f'Model {model} invalid')
        
        table = self.get_html_table_from_dict(coupon_dict, num_holes)
        if is_repair:
            col1, col2, col3 = colspan
            table = f"""
                <table>
                    <tr>
                        <th colspan="{col1}"></th>
                        <th id="summary" colspan="{col2}">Print</th>
                        <th id="summary" colspan="{col3}">Repair>/th>
                    </tr>
                    {table}
                </table>
            """
        else:
            table = f"""
            <table id="summary">
                {table}
            </table>
            """
        
        return table


    def get_applied_factor_html_table(self, model: str, is_repair: Optional[bool] = False) -> str:
        pass

    def get_matl_cf_html_table(self, is_repair: Optional[bool] = False) -> str:
        pass

    def get_report_html_contents(
            self, 
            model: str,
            section: int, 
            num_holes: int, 
            load_case: str, 
            repair: Optional[bool] = False,
            ) -> str:

        if repair:
            soln = 'repair'
            results = self.repair_batch_results[load_case]
        else:
            soln = 'print'
            results = self.print_batch_results[load_case]
        
        hole_num = lambda x, model: f'Hole {x}' if model in ('mlh', 'single-hole', 'lug') else {1: 'Central Hole', 2: 'Rivet-a', 3: 'Rivet-b'}[x] if model == 'satellite-hole' else 'Notch'
        ncols = 2
        nrows = num_holes // ncols
        if num_holes % 2:
            nrows += 1
        
        ey_html = []
        idx = 1
        for row in range(nrows):
            row_html = ['<tr>', ]
            for col in range(ncols):
                if idx <= num_holes:
                    if model in ('mlh', 'satellite-hole'):
                        ey_encoded = self.ctx.get_base64_png(f'./ey-{idx}-{soln}-{load_case}.png')
                    else:
                        ey_encoded = self.ctx.get_base64_png(f'./ey-{soln}-{load_case}.png')
                    
                    row_html.append(f'<tc><td><img src="{ey_encoded}" alt="ey {hole_num(idx, model)}" width="350" height="350"/><br><center>{hole_num(idx, model)}</center></td></tc>')
                idx += 1
            row_html.append('</tr>')
            ey_html.append(''.join(row_html))

        ey_html = f'<table>{"".join(ey_html)}</table>'
        s1_encoded = self.ctx.get_base64_png(f'./s1-{soln}-{load_case}.png')
        
        if results.get('ktdls', {}).get(0, {}).get('kckt', 0.0) > 1e-6:
            html = f"""
                <h4>{section}.1 Peak Stress</h4>
                {ey_html}
                <h4>{section}.2 K<sub>c</sub>/K<sub>t</sub></h4>
                <h4>{section}.3 Gradient Ratio</h4>
                <img src="{s1_encoded}" alt="{soln} s1 fringe plot" width="350" height="350"/>
                <h4>{section}.4 Shallow Gradient Factor</h4>
                <h4>{section}.5 Margin of Safety</h4>
                """
        else:
            html = f"""
                <h4>{section}.1 Peak Stress</h4>
                {ey_html}
                <h4>{section}.2 K<sub>c</sub>/K<sub>t</sub></h4>
                <h4>{section}.3 Gradient Ratio</h4>
                <img src="{s1_encoded}" alt="{soln} s1 fringe plot" width="350" height="350"/>
                """
        
        return html
    
    def export_webpage(self, web_view: QWebEngineView) -> None:
        from PyQt5.QtWidgets import QFileDialog

        dlg = QFileDialog(
            self,
            caption='Save Report as PDF',
        )
        file_name, _ = dlg.getSaveFileName(filter='*.pdf')

        page = web_view.page()
        page.printToPdf(file_name)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-4: truncated \uXXXX escape (Temp/ipykernel_9188/3160942269.py, line 329)

In [21]:
ctx = Context()
view = WebView(ctx)
view.show()
sys.exit(qapp.exec())

AttributeError: 'WebView' object has no attribute 'get_coupon_html_table'

In [1]:
from PyQt5.QtCore import Qt